# Learning Pytorch

This series would be taken from various tutorial available in youtube.

## 1 : Creating a simple network

This one is taken from excellent channel Alladin Perrson : https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

![](https://i.morioh.com/200620/5b0ea047.jpg)

### Imports

In [1]:
import torch.nn as  nn
import torch.nn.functional as F

In [2]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
import torch

In [4]:
from torch.utils.data import DataLoader

In [5]:
import torch.optim as optim

### Creating a fully connected network

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, classes)
    
    def forward(self,x):
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        return out
        

In [7]:
# testing
model = SimpleNN(784, 10)
input_tensor = torch.randn(64, 784)
out_tensor = model(input_tensor)
out_tensor.shape

torch.Size([64, 10])

### Setting the device

In [8]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Setting up hyperparameters

In [9]:
input_size = 784
classes = 10
learning_rate = 0.001
batch_size=64
num_epochs=100

In [10]:
train_dataset = datasets.MNIST(root='dataset/', download=True, train=True, transform=transforms.ToTensor())

0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


0it [00:00, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [12]:
test_dataset = datasets.MNIST(root='dataset/', download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialising the model

In [13]:
simple_model = SimpleNN(input_size,classes)

### Loss and optimiser

In [14]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

### Training the model

In [18]:
for epoch in range(num_epochs):
    current_loss  = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # reshape the data
        data = data.reshape(data.shape[0], -1)
        
        # calculate score and loss forward pass
        scores = model(data)
        loss = loss_criterion(scores, target)
        current_loss = loss
        
        # update weights backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # optimzer step
        optimizer.step()
    print(f"current epoch : {epoch} loss: {current_loss}")

current epoch : 0 loss: 0.0020287882070988417
current epoch : 1 loss: 0.002754819579422474
current epoch : 2 loss: 1.1342566722305492e-05
current epoch : 3 loss: 1.4342148233481566e-06
current epoch : 4 loss: 8.194585461751558e-06


KeyboardInterrupt: 

### Calculating the accuracy

In [21]:
def check_accuracy(model, loader):
    model.eval()
    num_samples = 0
    num_correct = 0
    
    with torch.no_grad():
        for x,y in loader:
            x = x.reshape(x.shape[0],-1)
            score = model(x)
            print(score)
            _,predictions = score.max(1)
            print(predictions, y)
            
            num_samples += predictions.size(0)
            num_correct += (y==predictions).sum()
            break
            
    print(f" total samples = {num_samples} , total correct = {num_correct}, percentage = {float(num_correct)/float(num_samples)*100:.2f}%")

In [22]:
check_accuracy(simple_model, train_loader)
check_accuracy(simple_model, test_loader)

tensor([[-0.0107,  0.0820,  0.1110, -0.0093,  0.0492, -0.0671,  0.0697,  0.1223,
          0.0715, -0.0019],
        [-0.2280, -0.0329,  0.0641, -0.0099,  0.0671, -0.0323,  0.1028,  0.2071,
          0.0762,  0.0555],
        [-0.0991,  0.0365,  0.1163,  0.1054,  0.0823, -0.0130,  0.0313,  0.1395,
         -0.0484,  0.1080],
        [ 0.0010, -0.0185,  0.0790,  0.1465,  0.1055,  0.0745,  0.0632,  0.1552,
          0.0132, -0.0210],
        [-0.0679, -0.0260,  0.1532,  0.0871,  0.0959,  0.0871, -0.0788,  0.1562,
          0.0789,  0.0834],
        [-0.0973, -0.0692,  0.1219, -0.0290,  0.1045,  0.0323,  0.1025,  0.2408,
         -0.0555, -0.0102],
        [ 0.0043, -0.0205,  0.1628,  0.0955,  0.0726,  0.0090, -0.0392,  0.0952,
          0.0428,  0.0372],
        [-0.2190, -0.0568,  0.0704,  0.1306,  0.1154, -0.0205,  0.0646,  0.3300,
         -0.0077, -0.0470],
        [-0.1645, -0.0127,  0.1136,  0.0706,  0.0310,  0.0371,  0.0530,  0.2666,
          0.0753, -0.0369],
        [-0.0473,  

The model seems to be very biased towards 7. I wonder why.